In [1]:
import torch
import torch.nn.functional as F
import torch_geometric as tg
from torch_geometric.nn import knn_graph, fps, knn
from torch_geometric.data import Data
import torch_geometric.io as tgio
from torch_geometric.io import read_off, read_ply
from torch_geometric.transforms import SamplePoints, NormalizeScale
import os, sys, time, random
import numpy as np
import pptk


In [3]:
dataset_path = '/data/pkurei/mpeg/dataset'
ply=os.path.join(dataset_path, 'longdress', 'Ply', 'longdress_vox10_1099.ply')
ply

'/data/pkurei/mpeg/dataset/longdress/Ply/longdress_vox10_1099.ply'

In [4]:
res=process_ply(ply_path=ply)

Data(face=[0], pos=[765821, 3])


In [6]:
res.shape

torch.Size([2, 77347921, 3])

In [22]:
from plyfile import PlyData, PlyElement

In [24]:
mesh=PlyData.read(ply)

In [53]:
mesh.elements[0].data['red'].shape

(824064,)

In [41]:
mesh.elements[0]

PlyElement('vertex', (PlyProperty('x', 'float'), PlyProperty('y', 'float'), PlyProperty('z', 'float'), PlyProperty('red', 'uchar'), PlyProperty('green', 'uchar'), PlyProperty('blue', 'uchar')), count=824064, comments=[])

In [45]:
mesh.elements[0].data

array([(231.,  14., 411., 124,  85,  64),
       (231.,  15., 411., 127,  86,  65),
       (231.,  13., 412., 118,  83,  63), ...,
       (320., 785., 513., 163, 150, 144),
       (320., 786., 512., 156, 146, 143),
       (320., 787., 512., 139, 130, 126)],
      dtype=[('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('red', 'u1'), ('green', 'u1'), ('blue', 'u1')])

In [48]:
a=torch.randn([1000,3])

In [50]:
a.to(torch.long).dtype

torch.int64

In [52]:
torch.float32

torch.float32

In [55]:
824064/2048

402.375

In [63]:
a=torch.from_numpy(mesh.elements[0].data['x'].copy())

In [64]:
a.shape

torch.Size([824064])

In [71]:
x=torch.randn([1000,3])

In [72]:
y=x[torch.randperm(len(x))[:10]]

In [74]:
y.shape

torch.Size([10, 3])

In [75]:
from torch_geometric.nn import knn

In [87]:
res=knn(x,y,k=11)
res.shape

torch.Size([2, 110])

In [86]:
torch.einsum('ijk->jki',res.reshape(2, 10, 11)).shape

torch.Size([10, 11, 2])

In [92]:
res[1].reshape([10,11])

tensor([[758, 447, 618, 662,  32, 311,  68, 633, 191, 403, 982],
        [257,  42, 592, 551, 765, 400, 933, 577, 568, 831, 176],
        [314, 266, 696, 452, 936, 464, 462, 676, 938, 587, 824],
        [423, 399,  78, 893, 601, 918, 748, 586, 486, 396, 267],
        [455, 581, 491, 126, 804, 177, 239, 494, 657, 782, 829],
        [260, 990, 331, 282, 912, 322, 989, 553, 469, 431, 156],
        [577, 592, 400,  60, 831, 701,  38, 933, 629,   8, 176],
        [648, 185, 759, 913, 181, 379, 868, 928, 799, 643, 355],
        [467, 548, 453,  52, 143, 680, 536, 876, 398,  15, 159],
        [180, 621, 461, 601, 937, 393, 483, 348, 739, 837, 893]])

In [4]:
np.floor(1.1)

1.0

In [7]:
for p in os.walk(os.path.join(dataset_path, 'longdress', 'Ply')):
    print(p)

('/data/pkurei/mpeg/dataset/longdress/Ply', [], ['longdress_vox10_1230.ply', 'longdress_vox10_1199.ply', 'longdress_vox10_1117.ply', 'longdress_vox10_1086.ply', 'longdress_vox10_1102.ply', 'longdress_vox10_1090.ply', 'longdress_vox10_1138.ply', 'longdress_vox10_1063.ply', 'longdress_vox10_1198.ply', 'longdress_vox10_1298.ply', 'longdress_vox10_1331.ply', 'longdress_vox10_1097.ply', 'longdress_vox10_1292.ply', 'longdress_vox10_1131.ply', 'longdress_vox10_1293.ply', 'longdress_vox10_1270.ply', 'longdress_vox10_1111.ply', 'longdress_vox10_1083.ply', 'longdress_vox10_1326.ply', 'longdress_vox10_1088.ply', 'longdress_vox10_1070.ply', 'longdress_vox10_1303.ply', 'longdress_vox10_1234.ply', 'longdress_vox10_1156.ply', 'longdress_vox10_1284.ply', 'longdress_vox10_1169.ply', 'longdress_vox10_1214.ply', 'longdress_vox10_1262.ply', 'longdress_vox10_1112.ply', 'longdress_vox10_1087.ply', 'longdress_vox10_1238.ply', 'longdress_vox10_1089.ply', 'longdress_vox10_1264.ply', 'longdress_vox10_1216.ply',